# Similarity and Bipartite Networks with Python and NetworkX
This notebook is an introduction to the concept and syntax around similarity and bipartite networks


In [ ]:
# Packaging
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_distances

sns.set(color_codes=True, rc={'figure.figsize':(10,8)})

# Adittional dataviz
#pip install -U nxviz -qq
import nxviz as nv

## Similarity networks

Can be constructed by mapping similarity between all observarions.
Here we are going to use cosine distances

In [2]:
# load cars data
data = pd.read_csv('https://gist.githubusercontent.com/ZeccaLehn/4e06d2575eb9589dbe8c365d61cb056c/raw/64f1660f38ef523b2a1a13be77b002b98665cdfe/mtcars.csv')

In [ ]:
data.head()


In [ ]:
# in dataset we have 32 cars with 12 variables (first is name)
data.shape

In [5]:
# Scale values

from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler()

data_num = scl.fit_transform(data.iloc[:,1:])

In [ ]:
data_num[0]

In [7]:
# Calculate distances into a square matrix
dist = cosine_distances(data_num,data_num)

In [ ]:
dist[0]

In [ ]:
pd.Series(dist.flatten()).hist()

In [ ]:
1-dist

In [11]:
# calculate a cutoff (for a less crowded network)
perc = np.percentile(1-dist, 60)

In [12]:
# create NW
G = nx.from_numpy_array(1-dist)

In [13]:
G.remove_edges_from(nx.selfloop_edges(G))

In [14]:
# add names

attributes_dict=data.iloc[:,0].T.to_dict()
nx.set_node_attributes(G, attributes_dict, 'model')

In [ ]:
print(G)

In [16]:
# Get rid of low-weight edges
G_sub = nx.edge_subgraph(G, [(u,v) for u,v,d in G.edges(data=True) if d['weight'] > perc])

In [ ]:
print(G_sub)

In [18]:
# identify communities (optional)
import community.community_louvain as community_louvain

partition = community_louvain.best_partition(G_sub)
nx.set_node_attributes(G_sub, partition, 'partition')

In [ ]:
nx.draw_kamada_kawai(G_sub,
               node_color=list(partition.values()),
               with_labels = True,
               labels=attributes_dict,
               font_color='r')

In [20]:
# For visualization
#!pip install -U bokeh -qq
#!pip install -q holoviews -qq

In [ ]:
# Import the libraries and link to the bokeh backend
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.plotting import show

# Setting the default figure size a bit larger
defaults = dict(width=750, height=750, padding=0.1,
                xaxis=None, yaxis=None)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

In [ ]:
graph = hv.Graph.from_networkx(G_sub, nx.layout.fruchterman_reingold_layout).opts(
                                                                        tools=['hover'],
                                                                        #directed=True,
                                                                        edge_alpha=0.2,
                                                                        #node_size='cent_degree',
                                                                        node_color='partition', cmap='Set1',
                                                                        legend_position='right'
                                                                        )

labels = hv.Labels(graph.nodes, ['x', 'y'], 'model')

show(hv.render((graph * labels.opts(text_font_size='8pt', text_color='black', bgcolor='white'))))

# Multi-modal networks


## What's that?
Now its time to talk about an interesting type of networks, multi-modal. This means, a network has several "modes", meaning connects entities on different conceptual levels. The most commone one is a **2-mode** (or **bipartite**) network.

Examples could be an

* Author $\rightarrow$ Paper
* Inventor $\rightarrow$ Patent
* Member $\rightarrow$ Club network.

Here, the elements in the different modes represent different things. In interesting real-life research examples you find 2-mode networks for instance in co-occurence (2 actors mentioned in the same news-article), co-affiliation (2 actors are member of the same association), or co-characteristics (2 actors both like to talk about a certain topic on twitter).

## Network Projections

Two-mode networks are rarely analysed in their original form. Although this is preferable, few methods exist for that purpose. As such, these networks are often transformed into one-mode networks (only one type of nodes) to be analysed. This procedure is often referred to as projection. Projection is done by selecting one of the sets of nodes and linking two nodes from that set if they were connected to the same node (of the other kind).

We can alalyse them in sepperation (and sometimes we should), but often its helpful to *project* them onto one mode. Here, we create a node in one mode by joint association with another mode.

2-mode

![](https://toreopsahl.files.wordpress.com/2009/04/fig1_twomode_half.png)

1-mode

![](https://toreopsahl.files.wordpress.com/2009/04/fig1_twomode_simple.png)

In my field, that often happens with scientometric data such as publications, but also patents or policy reports. Conceptually, we can see them as 2 mode networks, between articles and their reference.


![](https://www.dropbox.com/s/e4vnq7kh24pyu0t/networks_2mode.png?dl=1)

Particularly in citation networks, we can also use the implicite 2-mode structure of $Publications \rightarrow Citation$

That helps us to apply some interesting metrics, such as:

* direct citations
* Bibliographic coupling
* Co--citations

Interestingly, different projections of this 2-mode network give the whole resulting 1-mode network a different meaning.

![](https://www.dropbox.com/s/f8g8nr83lucvpqx/networks_biblio.png?dl=1)

For an application, check:

* Rakas, M., & Hain, D. S. (2019). The state of innovation system research: What happens beneath the surface?. Research Policy.



## Weighted Network Projection

In a similar spirit as the method used by Newman (2001), it is also possible to discount for the number of nodes when projecting weighted two-mode networks.


 For example, it could be argued that if many online users post to a thread, their ties should be weaker than if there were few people posting to the thread. A straight forward generalisation is the following function: $w_{ij} = \sum_p \frac{w_{i,p}}{N_p - 1}$.

 This formula would create a directed one-mode network in which the out-strength of a node is equal to the sum of the weights attached to the ties in the two-mode network that originated from that node. For example, node C has a tie with a weight of 5 in the two-mode network and an out-strength of 5 in the one-mode projection.

![](https://toreopsahl.files.wordpress.com/2009/04/fig1_twomode_forum_newman2001.png)

* Newman, M. E. J., 2001. Scientific collaboration networks. II. Shortest paths, weighted networks, and centrality. Physical Review E 64, 016132.

## Example

In [23]:
people = ['Jesper', 'Pernille', 'Morten', 'Lise', 'Christian', 'Mette', 'Casper', 'Dorte', 'Jacob', 'Helle']
places = ['Yoga House', 'Crossfit', 'Jazz Club', 'Jomfru Anne Gade', 'IsBjorn Sauna', 'Kajak klub', 'MusicHus']

In [24]:
# some more imports that will be useful
from networkx.algorithms import bipartite
import itertools
import random

In [25]:
# Creating a random bipartite network of people and places
combinations = list(itertools.product(people, places))
connections = random.sample(combinations, 20)

In [26]:
c0 = set([c[0] for c in connections])
c1 = set([c[1] for c in connections])

In [27]:
B = nx.Graph()

In [28]:
# add nodes and edges in their modes
B.add_nodes_from(c0, bipartite=0)
B.add_nodes_from(c1, bipartite=1)
B.add_edges_from(connections)

In [ ]:
# very clunky visualization of 2-mode networks (unfortunately)
l, r = nx.bipartite.sets(B)
pos = {}

# Update position for node from each group
pos.update((node, (1, index)) for index, node in enumerate(l))
pos.update((node, (2, index)) for index, node in enumerate(r))

nx.draw(B, pos=pos, with_labels=True)
plt.show()

In [30]:
# projecting onto people
B_people = bipartite.weighted_projected_graph(B, c0)

In [31]:
# projecting onto places
B_places = bipartite.weighted_projected_graph(B, c1)

In [ ]:
nx.draw(B_people, with_labels=True)

In [ ]:
nx.draw(B_places, with_labels=True)

## Network projection options

In [34]:
# pull edges
edges_df = nx.to_pandas_edgelist(B)

In [ ]:
edges_df

In [36]:
# create matrix from edges
adj_df = pd.crosstab(edges_df.source, edges_df.target)

In [ ]:
adj_df

In [ ]:
# Projecting with dot-product as alternative
pd.DataFrame(np.dot(adj_df, adj_df.T),
             index=adj_df.index,
             columns=adj_df.index)

In [ ]:
# with ns
nx.to_pandas_adjacency(B_people)

In [40]:
centrality_dgr = nx.degree_centrality(B_people)
centrality_eigen = nx.eigenvector_centrality_numpy(B_people, weight='weight')
centrality_between = nx.betweenness_centrality(B_people, weight='weight')

In [ ]:
# Sort the centrality dictionary from highest to lowest
sorted_centrality = sorted(centrality_dgr.items(), key=lambda x: x[1], reverse=True)

# Print the sorted centrality values
print("Degree Centrality (from highest to lowest):")
for node, centrality in sorted_centrality:
    print(f"{node}: {centrality:.4f}")

In [ ]:
# Sort the centrality dictionary from highest to lowest
sorted_centrality = sorted(centrality_eigen.items(), key=lambda x: x[1], reverse=True)

# Print the sorted centrality values
print("Eigenvector centrality (from highest to lowest):")
for node, centrality in sorted_centrality:
    print(f"{node}: {centrality:.4f}")

In [ ]:
# Sort the centrality dictionary from highest to lowest
sorted_between = sorted(centrality_between.items(), key=lambda x: x[1], reverse=True)

# Print the sorted centrality values
print("Eigenvector centrality (from highest to lowest):")
for node, centrality in sorted_between:
    print(f"{node}: {centrality:.4f}")